In [ ]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import glob
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd

# COCO tools
import pycocotools.mask
import rasterio.features
import shapely.geometry
import torch
import torchvision
import tqdm
import utils

# # Recognize Anything Model & Tag2Text
# from ram.models import ram_plus
# from ram import inference_ram
# import torchvision.transforms as TS
# Grounding Dino
from groundingdino.util.inference import load_model
from pycocotools.coco import COCO
from segment_anything import SamPredictor, sam_model_registry

# Segment anything
from segment_anything_hq import (
    SamPredictor as SamPredictor_hq,
)
from segment_anything_hq import (
    sam_model_registry as sam_model_registry_hq,
)

In [ ]:
# def load_image(image_path):
#     # load image
#     image_pil = PIL.Image.open(image_path).convert("RGB")  # load image

#     transform = T.Compose(
#         [
#             T.RandomResize([800], max_size=1333),
#             T.ToTensor(),
#             T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#         ]
#     )
#     image, _ = transform(image_pil, None)  # 3, h, w
#     return image_pil, image

# def get_grounding_output(model, image, caption, box_threshold, text_threshold,device="cpu"):
#     """

#     """
#     caption = caption.lower()
#     caption = caption.strip()
#     if not caption.endswith("."):
#         caption = caption + "."
#     model = model.to(device)
#     image = image.to(device)
#     with torch.no_grad():
#         outputs = model(image[None], captions=[caption])
#     logits = outputs["pred_logits"].cpu().sigmoid()[0]  # (nq, 256)
#     boxes = outputs["pred_boxes"].cpu()[0]  # (nq, 4)
#     logits.shape[0]

#     # filter output
#     logits_filt = logits.clone()
#     boxes_filt = boxes.clone()
#     filt_mask = logits_filt.max(dim=1)[0] > box_threshold
#     logits_filt = logits_filt[filt_mask]  # num_filt, 256
#     boxes_filt = boxes_filt[filt_mask]  # num_filt, 4
#     logits_filt.shape[0]

#     # get phrase
#     tokenlizer = model.tokenizer
#     tokenized = tokenlizer(caption)
#     # build pred
#     pred_phrases = []
#     scores = []
#     for logit, box in zip(logits_filt, boxes_filt):
#         pred_phrase = get_phrases_from_posmap(logit > text_threshold, tokenized, tokenlizer)
#         pred_phrases.append(pred_phrase + f"({str(logit.max().item())[:4]})")
#         scores.append(logit.max().item())
#     return boxes_filt, torch.Tensor(scores), pred_phrases


# def show_mask(mask, ax, random_color=False):
#     if random_color:
#         color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
#     else:
#         color = np.array([30/255, 144/255, 255/255, 0.6])
#     h, w = mask.shape[-2:]
#     mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     ax.imshow(mask_image)


# def show_box(box, ax, label):
#     x0, y0 = box[0], box[1]
#     w, h = box[2] - box[0], box[3] - box[1]
#     ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))
#     ax.text(x0, y0, label)


# def save_mask_data(output_dir, tags_chinese, mask_list, box_list, label_list):
#     value = 0  # 0 for background

#     mask_img = torch.zeros(mask_list.shape[-2:])
#     for idx, mask in enumerate(mask_list):
#         mask_img[mask.cpu().numpy()[0] == True] = value + idx + 1
#     plt.figure(figsize=(10, 10))
#     plt.imshow(mask_img.numpy())
#     plt.axis('off')
#     plt.savefig(os.path.join(output_dir, 'mask.jpg'), bbox_inches="tight", dpi=300, pad_inches=0.0)

#     json_data = {
#         'tags_chinese': tags_chinese,
#         'mask':[{
#             'value': value,
#             'label': 'background'
#         }]
#     }
#     for label, box in zip(label_list, box_list):
#         value += 1
#         name, logit = label.split('(')
#         logit = logit[:-1] # the last is ')'
#         json_data['mask'].append({
#             'value': value,
#             'label': name,
#             'logit': float(logit),
#             'box': box.numpy().tolist(),
#         })
#     with open(os.path.join(output_dir, 'label.json'), 'w') as f:
#         json.dump(json_data, f)

In [ ]:
# # Dino thresholds
# DINO_BOX_THRESHOLD = 0.2
# DINO_TEXT_THRESHOLD = 0.2

# # Boxes merge
# DO_IOU_MERGE = True
# IOU_THRESHOLD = 0.5

# # tiffbounds
# path_tifftiles = Path("data/tile_dataset")

# DEVICE = "cuda:1"

# fixed_tags = True
# RAM_CHECKPOINT = ".//ram_weights/ram_plus_swin_large_14m.pth"
# RAM_MODELTYPE = "swin_l"

# GROUNDING_DINO_CONFIG_PATH = "config/GroundingDINO_SwinB_cfg.py"
# GROUNDING_DINO_CHECKPOINT_PATH = "data/dino_weights/groundingdino_swinb_cogcoor.pth"


# USE_SAM_HQ = False
# SAM_ENCODER_VERSION = "vit_h"
# SAM_CHECKPOINT_PATH = "data/sam_weights/sam_vit_h_4b8939.pth"
# SAM_HQ_ENCODER_VERSION = "vit_h"
# SAM_HQ_CHECKPOINT_PATH = "data/sam_weights/sam_hq_vit_h.pth"

In [ ]:
# initialize Recognize Anything Model (RAM)
# normalize = TS.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# transform = TS.Compose([TS.Resize((384, 384)), TS.ToTensor(), normalize])
# ram_model = ram_plus(pretrained=RAM_CHECKPOINT, image_size=384, vit=RAM_MODELTYPE)
# pd.DataFrame(dict(ram_class=ram_model.tag_list, ram_threshold=ram_model.class_threshold)).to_csv("RAM_tag_list.csv")
# # ram_model.class_threshold = torch.ones(ram_model.num_class) * RAM_THRESHOLD

# ram_model.eval()
# ram_model = ram_model.to(DEVICE)


from hydra import compose, initialize

with initialize("config", version_base=None):
    cfg = compose("config.yaml")

print(cfg)
# Load grounding dino model
dino_model = load_model(
    cfg.GROUNDING_DINO_CONFIG_PATH,
    cfg.GROUNDING_DINO_CHECKPOINT_PATH,
    device=cfg.DEVICE,
)

# Segment Anything Model (SAM)

if cfg.USE_SAM_HQ:
    print("Initialize SAM-HQ Predictor")
    sam = sam_model_registry_hq[cfg.SAM_HQ_ENCODER_VERSION](
        checkpoint=cfg.SAM_HQ_CHECKPOINT_PATH
    ).to(device=cfg.DEVICE)
    sam_predictor = SamPredictor_hq(sam)
else:
    sam = sam_model_registry[cfg.SAM_ENCODER_VERSION](
        checkpoint=cfg.SAM_CHECKPOINT_PATH
    ).to(device="cuda:1")
    sam_predictor = SamPredictor(sam)

In [ ]:
def find_tile_bounds(root_tilepath, concat=True):
    """
    Recursively search through directories starting from root_tilepath to find and optionally concatenate
    geospatial data files (.gpkg) that include 'cells_intersect' in their filename.

    Parameters:
    - root_tilepath (str or pathlib.Path): The root directory path where the search for tile files begins.
    - concat (bool, optional): If True, concatenates all found geospatial data into a single DataFrame.
                               If False, returns a list of DataFrames. Default is True.

    Returns:
    - pandas.DataFrame or list of geopandas.GeoDataFrame: The concatenated DataFrame of all files if `concat=True`,
      or a list of DataFrames for each file if `concat=False`.

    """
    cell_files = []
    for p in Path(str(root_tilepath)).iterdir():
        if p.is_dir():
            cell_files += find_tile_bounds(p, concat=False)
        elif p.is_file() and p.suffix == ".gpkg" and "tiles_intersect" in p.stem:
            df = gpd.read_file(p)
            df["name"] = p.stem.replace("_tiles_intersects", "")
            cell_files.append(df.copy())

    if concat:
        cell_files = pd.concat(cell_files)

    return cell_files


# tilebounds
df_tilebounds = find_tile_bounds(Path(cfg.disk_path) / "tile_dataset")

In [ ]:
df_tilebounds

In [ ]:
out_dir = Path(cfg.disk_path) / "output"
if not out_dir.exists():
    out_dir.mkdir()
else:
    print(f"Directory {out_dir} already exists!")

In [ ]:
supervisely_path = Path(cfg.disk_path) / "supervisely" / "2023*"
dataDirs = [Path(p) for p in glob.glob(str(supervisely_path))]

for dataDir in dataDirs:
    # initialize COCO api for instance annotations
    coco = COCO(dataDir.joinpath("annotations/instances.json"))
    cats = coco.loadCats(coco.getCatIds())
    nms = [cat["name"] for cat in cats]
    catIds = coco.getCatIds(catNms=nms)
    imgIds = coco.getImgIds()

    # fig = plt.figure(figsize=(20, 10), dpi=100)

    df_pred_shapes = dict(
        category=[],
        confidence=[],
        tile_path=[],
        project_name=[],
        tile_fname=[],
        geometry=[],
    )
    df_seg_shapes = dict(
        category=[], tile_path=[], tile_fname=[], project_name=[], geometry=[]
    )

    projName = (
        Path(glob.glob(f"{str(dataDir)}/images/*")[0])
        .stem.split("_")[0]
        .replace(" ", "_")
    )

    for imgId in tqdm.tqdm(imgIds):
        # COCO format image path and annotation
        img = coco.loadImgs(imgId)[0]
        imgPath = dataDir.joinpath("images").joinpath(img["file_name"])
        anns = {}
        for catId in catIds:
            anns[catId] = coco.loadAnns(
                coco.getAnnIds(imgIds=img["id"], catIds=[catId], iscrowd=None)
            )

        # Load image
        image_pil, image = utils.load_image(imgPath)

        # Tags
        if cfg.fixed_tags:
            tags = ",".join(cfg.fixed_tags)
        else:
            # # Find tags with RAM
            # ram_model = ram_model.to(cfg.DEVICE)
            # raw_image = image_pil.resize((384, 384))
            # raw_image = transform(raw_image).unsqueeze(0).to(cfg.DEVICE)
            # res = inference_ram(raw_image, ram_model)
            # tags = res[0].replace(" |", ",")
            raise NotImplementedError("RAM not implemented")

        # Find bounding boxes with grounding dino
        boxes_filt, scores, pred_phrases = utils.get_grounding_output(
            dino_model,
            image,
            tags,
            cfg.DINO_BOX_THRESHOLD,
            cfg.DINO_TEXT_THRESHOLD,
            device=cfg.DEVICE,
        )

        # Resize boxes
        size = image_pil.size
        H, W = size[1], size[0]
        for i in range(boxes_filt.size(0)):
            boxes_filt[i] = boxes_filt[i] * torch.Tensor([W, H, W, H]).to(cfg.device)
            boxes_filt[i][:2] -= boxes_filt[i][2:] / 2
            boxes_filt[i][2:] += boxes_filt[i][:2]

        # use NMS to handle overlapped boxes
        boxes_filt = boxes_filt.cpu()
        nms_idx = (
            torchvision.ops.nms(boxes_filt, scores, cfg.IOU_THRESHOLD).numpy().tolist()
        )
        if cfg.DO_IOU_MERGE:
            boxes_filt_clean = boxes_filt[nms_idx]
            pred_phrases_clean = [pred_phrases[idx] for idx in nms_idx]
            # print(f"NMS: before {boxes_filt.shape[0]} boxes, after {boxes_filt_clean.shape[0]} boxes")
        else:
            boxes_filt_clean = boxes_filt
            pred_phrases_clean = pred_phrases

        # Segment objects with SAM
        image_np = np.array(image_pil)
        sam_predictor.set_image(image_np)
        transformed_boxes = sam_predictor.transform.apply_boxes_torch(
            boxes_filt_clean, image_np.shape[:2]
        ).to(cfg.DEVICE)
        masks, _, _ = sam_predictor.predict_torch(
            point_coords=None,
            point_labels=None,
            boxes=transformed_boxes.to(cfg.DEVICE),
            multimask_output=False,
        )

        # Find tile bounds (X, Y) based on name
        imgName = ".".join(coco.imgs[imgId]["file_name"].split(".")[:-1])
        tile1a = imgName.split("_")[0]
        tile1b = int(imgName.split("_")[-1])
        cellfile = df_tilebounds[
            (df_tilebounds.index == tile1b) & (df_tilebounds.name == tile1a)
        ].copy()
        assert len(cellfile) == 1
        cellfile = cellfile.iloc[0, :].copy()
        xstep = (cellfile.xmax - cellfile.xmin) / image_np.shape[1]
        ystep = (cellfile.ymax - cellfile.ymin) / image_np.shape[0]

        affine = [xstep, 0, cellfile.xmin, 0, -ystep, cellfile.ymax, 0, 0, 1]

        # SAM masks
        assert len(pred_phrases_clean) == len(masks)
        shapes, titles = [], []
        for cat_title, mask in zip(pred_phrases_clean, masks):
            mask = mask.cpu().numpy()
            cat_shapes = rasterio.features.shapes(
                mask.astype(np.uint8), mask=mask, connectivity=4, transform=affine
            )
            for shape, _ in cat_shapes:
                title, confidence = cat_title.replace(")", "").split("(")
                shape = shapely.geometry.shape(shape).simplify(
                    0.01, preserve_topology=True
                )
                if shape.area > 0.01:
                    df_pred_shapes["category"].append(title)
                    df_pred_shapes["confidence"].append(confidence)
                    df_pred_shapes["geometry"].append(shape)
                    df_pred_shapes["tile_path"].append(str(imgPath))
                    df_pred_shapes["tile_fname"].append(imgPath.stem)
                    df_pred_shapes["project_name"].append(projName)

        # COCO masks
        for catId, catName in zip(catIds, nms):
            for ann in anns[catId]:
                t = coco.imgs[ann["image_id"]]

                if isinstance(ann["segmentation"]["counts"], list):
                    rle = pycocotools.mask.frPyObjects(
                        [ann["segmentation"]], t["height"], t["width"]
                    )
                else:
                    rle = [ann["segmentation"]]

                m = pycocotools.mask.decode(rle)[:, :, 0]
                cat_shapes = rasterio.features.shapes(
                    m.astype(np.uint8), mask=m, connectivity=4, transform=affine
                )

                for shape, _ in cat_shapes:
                    title, confidence = cat_title.replace(")", "").split("(")
                    shape = shapely.geometry.shape(shape).simplify(
                        0.01, preserve_topology=True
                    )
                    if shape.area > 0.01:
                        df_seg_shapes["category"].append(catName)
                        df_seg_shapes["geometry"].append(shape)

                        df_seg_shapes["tile_path"].append(str(imgPath))
                        df_seg_shapes["tile_fname"].append(imgPath.stem)
                        df_seg_shapes["project_name"].append(projName)

        # # Clear figure and add axis
        # fig.clear()
        # axs = fig.subplots(1, 2, squeeze=False)

        # # Draw supervisely image (COCO format)
        # axs[0,0].imshow(image_np)
        # i_end = 0
        # legend_handles = []
        # for catId, catName in zip(catIds, nms):
        #     plt.sca(axs[0,0])
        #     coco.showAnns(anns[catId], draw_bbox=True)
        #     for ann in anns[catId]:
        #         plt.text(ann["bbox"][0], ann["bbox"][1] - 5, catName, fontsize=10)

        # # draw RAM-Dino-SAM image
        # axs[0,1].imshow(image_np)
        # for mask in masks:
        #     show_mask(mask.cpu().numpy(), axs[0,1], random_color=True)

        # for box, label in zip(boxes_filt_clean, pred_phrases_clean):
        #     show_box(box.numpy(), axs[0,1], label)

        # # Set titles
        # axs[0,0].set_title("Supervisely")
        # axs[0,1].set_title(f"RAM-DINO-SAM, tags: {tags}", wrap=True)

        # # Dont show axis
        # axs[0,0].axis("off")
        # axs[0,1].axis("off")

        # # Save figure
        # if fixed_tags:
        #     if USE_SAM_HQ:
        #         plt_dir = pathlib.Path("test_output/fixed_tags_hq")
        #     else:
        #         plt_dir = pathlib.Path("test_output/fixed_tags")
        # else:
        #     if USE_SAM_HQ:
        #         plt_dir = pathlib.Path("test_output/ram_tags_hq")
        #     else:
        #         plt_dir = pathlib.Path("test_output/ram_tags")
        # plt_dir.mkdir(parents=True, exist_ok=True)
        # fig.savefig(plt_dir.joinpath(imgPath.name), bbox_inches="tight")

    df_pred_shapes = gpd.GeoDataFrame(df_pred_shapes, crs="epsg:28992")
    if cfg.USE_SAM_HQ:
        df_pred_shapes.to_file(out_dir / f"/fix_tags_hq_{projName}.gpkg")
    else:
        df_pred_shapes.to_file(out_dir / f"fix_tags_{projName}.gpkg")

    df_seg_shapes = gpd.GeoDataFrame(df_seg_shapes, crs="epsg:28992")
    df_seg_shapes.to_file(out_dir / f"supervisely_tags_{projName}.gpkg")

In [ ]:
df_seg_shapes